# 1. Título: Aeronaves - Drones cadastrados

## 2. Membros
- Danilo Pimentel de Carvalho Costa - 2016058077
- Dione Jessé Mendes Silva - 2014132130
- Gabriel Lucas Freire Martins e Silva - 2017023250
- Ícaro Kened Torres Neto - 2016077195

## 3. Descrição dos dados
O grupo escolheu o conjunto de dados relacionados ao cadastro de drones, feito pela ANAC. Os dados podem ser encontrados nesta URL: http://dados.gov.br/dataset/aeronaves-drones-cadastrados.

## 4. Diagrama ER
![Diagrama ER](diagrama-er.png "Diagrama ER")

## 5. Diagrama relacional
![Diagrama relacional](diagrama-relacional.png "Diagrama relacional")

## 6. Consultas

In [96]:
FILE_ENCODING = "ISO-8859-1"
FILE_NAME = "SISANT.csv"
FILE_URL = 'http://sistemas.anac.gov.br/dadosabertos/Aeronaves/drones%20cadastrados/SISANT.csv'

import pandas as pd
import requests
from io import BytesIO, StringIO
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from pathlib import Path
import codecs

print('Step 1 - Reading data')

# If the source file is not present, then request from the data source
if not Path(FILE_NAME).is_file():
    print('Disabling insecure request warnings...')
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    print('Requesting data from source...')
    r = requests.get(FILE_URL, verify=False)
    print('Writing data to file...')
    f = codecs.open(FILE_NAME, "w", FILE_ENCODING)
    f.write(r.content)
    f.close()

print('Reading file content...')
content = Path(FILE_NAME).read_text(encoding = ENCODING)

print('Creating dataframe...')
df = pd.read_csv(StringIO(content[23:]))

print('Step 1 - Done!')

Step 1 - Reading data
Reading file content...
Creating dataframe...
Step 1 - Done!


In [97]:
def make_dataframe(main_df, main_df_columns, new_column_names):
    new_df = main_df[main_df_columns].drop_duplicates()
    new_df.columns = new_column_names
    return new_df

print('Step 2 - Creating dataframes...')
            
print('Creating dataframe "operador"...')
df_operador = make_dataframe(
    df,
    ["CPF/CNPJ", "Operador", "Cidade-Estado"],
    ["cpf_cnpj", "nome", "cidade_estado"]
)
print('Done!\n')

print('Creating dataframe "licenca"...')
df_licenca = make_dataframe(
    df,
    ["CPF/CNPJ", "Número de serie", "Tipo Uso", "Data Validade"],
    ["cpf_cnpj", "n_serie", "tipo_uso", "data_validade"]
)
print('Done!\n')

print('Creating dataframe "drone"...')
df_drone = make_dataframe(
    df,
    ["Número de serie", "Peso máximo de decolagem (Kg)", "Ramo de atividade", "Modelo"],
    ["n_serie", "peso_max", "ramo", "modelo"]
)
print('Done!\n')

print('Creating dataframe "ramo"...')
df_ramo = make_dataframe(
    df,
    ["Ramo de atividade"],
    ["ramo"],
)
print('Done!\n')

print('Creating dataframe "modelo"...')
df_modelo = make_dataframe(
    df,
    ["Modelo", "Fabricante"],
    ["modelo", "fabricante"]
)
print('Done!\n')

print('Creating dataframe "fabricante"...')
df_fabricante = make_dataframe(
    df,
    ["Fabricante"],
    ["fabricante"]
)
print('Done!\n')

print('Step 2 - Done!')

Step 2 - Creating dataframes...
Creating dataframe "operador"...
Done!

Creating dataframe "licenca"...
Done!

Creating dataframe "drone"...
Done!

Creating dataframe "ramo"...
Done!

Creating dataframe "modelo"...
Done!

Creating dataframe "fabricante"...
Done!

Step 2 - Done!


In [98]:
def make_1_n_relationship(df1, df2, rel_column_name, match_column_name):
    df1[rel_column_name] = -1
    for index, row in df1.iterrows():
        rel_loc = df2.loc[df2[match_column_name] == row[match_column_name]]
        if not rel_loc.empty:
            df1.at[index, rel_column_name] = rel_loc.index[0]
    del df1[match_column_name]

print('Step 3 - Creating relationships...')

print('Making relationship drone-ramo...')
make_1_n_relationship(df_drone, df_ramo, "ramo_id", "ramo")
print('Done!\n')

print('Making relationship drone-modelo...')
make_1_n_relationship(df_drone, df_modelo, "modelo_id", "modelo")
print('Done!\n')

print('Making relationship modelo-fabricante...')
make_1_n_relationship(df_modelo, df_fabricante, "fabricante_id", "fabricante")
print('Done!\n')

print('Step 3 - Done!')

Step 3 - Creating relationships...
Making relationship drone-ramo...
Done!

Making relationship drone-modelo...
Done!

Making relationship modelo-fabricante...
Done!

Step 3 - Done!


In [99]:
def make_id_column(dataframe, id_column_name):
    dataframe[id_column_name] = -1
    for index, row in dataframe.iterrows():
        dataframe.at[index, id_column_name] = index

print('Step 4 - Creating ids and renaming columns...')

print('Creating ids for "ramo"...')
make_id_column(df_ramo, "id")
print('Done!\n')

print('Creating ids for "modelo"...')
make_id_column(df_modelo, "id")
print('Done!\n')

print('Creating ids for "fabricante"...')
make_id_column(df_fabricante, "id")
print('Done!\n')

print('Renaming columns for "ramo"...')
df_ramo = df_ramo[["id", "ramo"]]
df_ramo.columns = ["id", "nome"]
print('Done!\n')

print('Renaming columns for "modelo"...')
df_modelo = df_modelo[["id", "modelo", "fabricante_id"]]
df_modelo.columns = ["id", "nome", "fabricante_id"]
print('Done!\n')

print('Renaming columns for "fabricante"...')
df_fabricante = df_fabricante[["id", "fabricante"]]
df_fabricante.columns = ["id", "nome"]
print('Done!\n')

print('Step 4 - Done!')

Step 4 - Creating ids and renaming columns...
Creating ids for "ramo"...
Done!

Creating ids for "modelo"...
Done!

Creating ids for "fabricante"...
Done!

Renaming columns for "ramo"...
Done!

Renaming columns for "modelo"...
Done!

Renaming columns for "fabricante"...
Done!

Step 4 - Done!


In [100]:
DB_NAME = "tp2.sqlite"

import sqlite3

def toggle_foreign_keys(enable = True):
    global DB_NAME
    conn = sqlite3.connect(DB_NAME)
    conn.execute("PRAGMA foreign_keys = %d" % (1 if enable else 0))
    conn.commit()
    conn.close()
    
def create_table(tbl_name, fields):
    global DB_NAME
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()                               
    cur.execute("drop table if exists %s" % tbl_name)
    cur.execute("create table %s (%s)" % (tbl_name, ','.join(fields)))
    conn.commit()
    conn.close()
    
def populate_table(tbl_name, dataframe):
    global DB_NAME
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()                              
    wildcards = ','.join(['?'] * len(dataframe.columns))              
    data = [tuple(x) for x in dataframe.values]
    cur.executemany("insert into %s values(%s)" % (tbl_name, wildcards), data)
    conn.commit()
    conn.close()

print('Step 5 - Saving to db\n')

print('Creating table "operador"...')
create_table("operador", [
    "cpf_cnpj VARCHAR(50)",
    "nome VARCHAR(200)",
    "cidade_estado VARCHAR(50)",
    "PRIMARY KEY (cpf_cnpj)"
])
print('Done!\n')

print('Creating table "fabricante"...')
create_table("fabricante", [
    "id INT AUTO INCREMENT",
    "nome VARCHAR(200)",
    "PRIMARY KEY (id)"
])
print('Done!\n')

print('Creating table "modelo"...')
create_table("modelo", [
    "id INT AUTO INCREMENT",
    "nome VARCHAR(200)",
    "fabricante_id INT",
    "PRIMARY KEY (id)",
    "FOREIGN KEY (fabricante_id) REFERENCES fabricante(id)"
])
print('Done!\n')

print('Creating table "ramo"...')
create_table("ramo", [
    "id INT AUTO INCREMENT",
    "nome VARCHAR(200)",
    "PRIMARY KEY (id)"
])
print('Done!\n')

print('Creating table "drone"...')
create_table("drone", [
    "n_serie VARCHAR(50)",
    "peso_max FLOAT",
    "ramo_id INT",
    "modelo_id INT",
    "PRIMARY KEY (n_serie)",
    "FOREIGN KEY (ramo_id) REFERENCES ramo(id)",
    "FOREIGN KEY (modelo_id) REFERENCES modelo(id)"
])
print('Done!\n')

print('Creating table "licenca"...')
create_table("licenca", [
    "cpf_cnpj VARCHAR(50)",
    "n_serie VARCHAR(50)",
    "tipo_uso VARCHAR(50) NOT NULL",
    "data_validade DATE NOT NULL",
    "PRIMARY KEY (cpf_cnpj, n_serie)",
    "FOREIGN KEY (cpf_cnpj) REFERENCES operador(cpf_cnpj)",
    "FOREIGN KEY (n_serie) REFERENCES drone(n_serie)"
])
print('Done!\n')

toggle_foreign_keys(False)

print('Populating table "operador"...')
populate_table("operador", df_operador);
print('Done!\n')

print('Populating table "fabricante"...')
populate_table("fabricante", df_fabricante);
print('Done!\n')

print('Populating table "modelo"...')
populate_table("modelo", df_modelo);
print('Done!\n')

print('Populating table "ramo" ...')
populate_table("ramo", df_ramo);
print('Done!\n')

print('Populating table "drone"...')
populate_table("drone", df_drone);
print('Done!\n')

print('Populating table "licenca"...')
populate_table("licenca", df_licenca);
print('Done!\n')

toggle_foreign_keys(True)

print('Step 5 - Done!')

Step 5 - Saving to db

Creating table "operador"...
Done!

Creating table "fabricante"...
Done!

Creating table "modelo"...
Done!

Creating table "ramo"...
Done!

Creating table "drone"...
Done!

Creating table "licenca"...
Done!

Populating table "operador"...


IntegrityError: UNIQUE constraint failed: operador.cpf_cnpj

### 6.1 Duas consultas envolvendo seleção e projeção

#### 6.1.1 Consulta 1 (duas versões)

#### 6.1.2 Consulta 2 (duas versões)

### 6.2 Três consultas envolvendo junção de duas relações

#### 6.2.1 Consulta 3 (duas versões)

#### 6.2.2 Consulta 4 (duas versões)

#### 6.2.3 Consulta 5 (duas versões)

### 6.3 Três consultas envolvendo junção de três ou mais relações

#### 6.3.1 Consulta 6 (duas versões)

#### 6.3.2 Consulta 7 (duas versões)

#### 6.3.3 Consulta 8 (duas versões)

### 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

#### 6.4.1 Consulta 9 (duas versões)

#### 6.4.2 Consulta 10 (duas versões)

## 7. Autoavaliação dos membros